<a href="https://colab.research.google.com/github/yima-t/LastfmListens/blob/main/Copy_of_lastfm_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we write a function to collect Data from the last.fm Api endpoint.
The endpoint being used is the GetRecentTracks endpoint to get all the tracks scrobbled

In [ ]:
import requests as rq

API_KEY = 'aa67bd14bbef2d0cc14a3bfaaa10517f'
USER_AGENT = 'YTJ'
user='theiratebandit'

def lastfm_get(payload):
    headers= {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    payload['limit'] = 200
    payload['user']= user
    payload['api_key'] = API_KEY
    payload['format'] = 'json'


    response = rq.get(url,headers=headers, params=payload)
    return response
!pip install requests_cache
import requests_cache
requests_cache.install_cache()

import time
from IPython.core.display import clear_output

responses = []

page = 1
total_pages = 99999 # this is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'user.getrecenttracks',
        'limit': 200,
        'user': user,
        'page': page
    }

    # print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # clear the output to make things neater
    clear_output(wait = True)

    # make the API call
    response = lastfm_get(payload)

    # if we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # extract pagination info
    page = int(response.json()['recenttracks']['@attr']['page'])
    total_pages = int(response.json()['recenttracks']['@attr']['totalPages'])

    # append response
    responses.append(response)

    # if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # increment the page number
    page += 1

import pandas as pd

r0 = responses[0]
r0_json = r0.json()
r0_tracks = r0_json['recenttracks']['track']
r0_df = pd.DataFrame(r0_tracks)
r0_df.head()

frames = [pd.DataFrame(r.json()['recenttracks']['track']) for r in responses]
tracks = pd.concat(frames)
tracks.head()

print(tracks.columns)

tracks.head()

Index(['artist', 'streamable', 'image', 'mbid', 'album', 'name', 'url',
       'date'],
      dtype='object')


,artist,streamable,image,mbid,album,name,url,date
0,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Pray For Paris'}",$500 Ounces (feat. Freddie Gibbs & Roc Marciano),https://www.last.fm/music/Westside+Gunn/_/$500...,"{'uts': '1705186913', '#text': '13 Jan 2024, 2..."
1,"{'mbid': '', '#text': 'Westside Gunn'}",0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Hall & Nash 2'}",Fuck & Get High,https://www.last.fm/music/Westside+Gunn/_/Fuck...,"{'uts': '1705186705', '#text': '13 Jan 2024, 2..."
2,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,{'mbid': 'd9972546-40c4-4c3e-be9f-8e9779ea59a2...,Why I do em Like that (feat. Billie Essco),https://www.last.fm/music/Westside+Gunn/_/Why+...,"{'uts': '1705186464', '#text': '13 Jan 2024, 2..."
3,{'mbid': 'a536dd07-131b-48ef-ab47-141b752ba082...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,{'mbid': '2488e0ce-7a41-46a2-b1dc-a3fefe38b7d0...,"The Rivington (feat. Conway, Westside Gunn & J...",https://www.last.fm/music/Free+Nationals/_/The...,"{'uts': '1705173308', '#text': '13 Jan 2024, 1..."
4,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,{'mbid': '69f96a35-ddb2-4e26-83e0-2bd6c43776df...,Mamas PrimeTime,https://www.last.fm/music/Westside+Gunn/_/Mama...,"{'uts': '1705173001', '#text': '13 Jan 2024, 1..."


Cleaning the data by dropping columns that are not needed

In [ ]:
tracks=tracks.drop(['mbid','streamable'], axis=1)

Creating a new dataframe that contains only the distinct tracks so we can use the track name to get the top tags from each track using another endpoint

In [ ]:
tracksdistinct= tracks.drop_duplicates(subset=["url"])
tracksdistinct.describe()

,artist,image,album,name,url,date
count,5835,5835,5835,5835,5835,5835
unique,1721,2601,2664,5742,5835,5739
top,{'mbid': 'e0e1db18-f7ba-4dee-95ff-7ae8cf545460...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': '742362af-2bca-4e41-9875-f96c855d2b92...,Intro,https://www.last.fm/music/Westside+Gunn/_/$500...,"{'uts': '1641557302', '#text': '07 Jan 2022, 1..."
freq,137,71,28,6,1,2


In [ ]:
tracksdistinct=tracksdistinct.rename(columns={'name':'track'})
tracksdistinct.head()

,artist,image,album,track,url,date
0,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'mbid': '', '#text': 'Pray For Paris'}",$500 Ounces (feat. Freddie Gibbs & Roc Marciano),https://www.last.fm/music/Westside+Gunn/_/$500...,"{'uts': '1705186913', '#text': '13 Jan 2024, 2..."
1,"{'mbid': '', '#text': 'Westside Gunn'}","[{'size': 'small', '#text': 'https://lastfm.fr...","{'mbid': '', '#text': 'Hall & Nash 2'}",Fuck & Get High,https://www.last.fm/music/Westside+Gunn/_/Fuck...,"{'uts': '1705186705', '#text': '13 Jan 2024, 2..."
2,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': 'd9972546-40c4-4c3e-be9f-8e9779ea59a2...,Why I do em Like that (feat. Billie Essco),https://www.last.fm/music/Westside+Gunn/_/Why+...,"{'uts': '1705186464', '#text': '13 Jan 2024, 2..."
3,{'mbid': 'a536dd07-131b-48ef-ab47-141b752ba082...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': '2488e0ce-7a41-46a2-b1dc-a3fefe38b7d0...,"The Rivington (feat. Conway, Westside Gunn & J...",https://www.last.fm/music/Free+Nationals/_/The...,"{'uts': '1705173308', '#text': '13 Jan 2024, 1..."
4,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': '69f96a35-ddb2-4e26-83e0-2bd6c43776df...,Mamas PrimeTime,https://www.last.fm/music/Westside+Gunn/_/Mama...,"{'uts': '1705173001', '#text': '13 Jan 2024, 1..."


In [ ]:
tracksdistinct['artist'] = tracksdistinct['artist'].apply(lambda x: x['#text'])
tracksdistinct['album'] = tracksdistinct['album'].apply(lambda x: x['#text'])
tracksdistinct['date'] = tracksdistinct['date'].apply(lambda x: x['#text'])
tracksdistinct.head()

,artist,image,album,track,url,date
0,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Pray For Paris,$500 Ounces (feat. Freddie Gibbs & Roc Marciano),https://www.last.fm/music/Westside+Gunn/_/$500...,"13 Jan 2024, 23:01"
1,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hall & Nash 2,Fuck & Get High,https://www.last.fm/music/Westside+Gunn/_/Fuck...,"13 Jan 2024, 22:58"
2,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hitler Wears Hermes 8: Side B,Why I do em Like that (feat. Billie Essco),https://www.last.fm/music/Westside+Gunn/_/Why+...,"13 Jan 2024, 22:54"
3,Free Nationals,"[{'size': 'small', '#text': 'https://lastfm.fr...",Free Nationals,"The Rivington (feat. Conway, Westside Gunn & J...",https://www.last.fm/music/Free+Nationals/_/The...,"13 Jan 2024, 19:15"
4,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",And Then You Pray for Me,Mamas PrimeTime,https://www.last.fm/music/Westside+Gunn/_/Mama...,"13 Jan 2024, 19:10"


In [ ]:
def lookup_tags(artist,track):
    responser = lastfm_get({
        'method': 'track.getTopTags',
        'artist':  artist,
        'track': track
    })

    # if there's an error, just return nothing
    if responser.status_code != 200:
        return None

    # extract the top three tags and turn them into a string
    tags = [t['name'] for t in responser.json()['toptags']['tag'][:3]]
    tags_str = ', '.join(tags)

    # rate limiting
    if not getattr(responser, 'from_cache', False):
        time.sleep(0.25)
    return tags_str

from tqdm import tqdm
tqdm.pandas()

tracksdistinct['tags'] = tracksdistinct[['artist', 'track']].progress_apply(lambda row: lookup_tags(row['artist'], row['track']), axis=1)

tracksdistinct.head()

100%|██████████| 5835/5835 [1:22:46<00:00,  1.17it/s]


,artist,image,album,track,url,date,tags
0,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Pray For Paris,$500 Ounces (feat. Freddie Gibbs & Roc Marciano),https://www.last.fm/music/Westside+Gunn/_/$500...,"13 Jan 2024, 23:01","MySpotigramBot, underground hip-hop, abstract ..."
1,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hall & Nash 2,Fuck & Get High,https://www.last.fm/music/Westside+Gunn/_/Fuck...,"13 Jan 2024, 22:58",
2,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hitler Wears Hermes 8: Side B,Why I do em Like that (feat. Billie Essco),https://www.last.fm/music/Westside+Gunn/_/Why+...,"13 Jan 2024, 22:54","MySpotigramBot, -1001854528204"
3,Free Nationals,"[{'size': 'small', '#text': 'https://lastfm.fr...",Free Nationals,"The Rivington (feat. Conway, Westside Gunn & J...",https://www.last.fm/music/Free+Nationals/_/The...,"13 Jan 2024, 19:15",
4,Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",And Then You Pray for Me,Mamas PrimeTime,https://www.last.fm/music/Westside+Gunn/_/Mama...,"13 Jan 2024, 19:10",


In [ ]:
tracksdistinct.dtypes

artist    object
image     object
album     object
track     object
url       object
date      object
tags      object
dtype: object

In [ ]:
#joining the dataframes

full_tracks=pd.merge(tracks, tracksdistinct, on='url', how='left')
print(full_tracks.head())

                                            artist_x  \
0  {'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...   
1             {'mbid': '', '#text': 'Westside Gunn'}   
2  {'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...   
3  {'mbid': 'a536dd07-131b-48ef-ab47-141b752ba082...   
4  {'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...   

                                             image_x  \
0  [{'size': 'small', '#text': 'https://lastfm.fr...   
1  [{'size': 'small', '#text': 'https://lastfm.fr...   
2  [{'size': 'small', '#text': 'https://lastfm.fr...   
3  [{'size': 'small', '#text': 'https://lastfm.fr...   
4  [{'size': 'small', '#text': 'https://lastfm.fr...   

                                             album_x  \
0            {'mbid': '', '#text': 'Pray For Paris'}   
1             {'mbid': '', '#text': 'Hall & Nash 2'}   
2  {'mbid': 'd9972546-40c4-4c3e-be9f-8e9779ea59a2...   
3  {'mbid': '2488e0ce-7a41-46a2-b1dc-a3fefe38b7d0...   
4  {'mbid': '69f96a35-ddb2-4e26-83e0-2bd6c4377

In [ ]:
full_tracks.head()

,artist_x,image_x,album_x,name,url,date_x,artist_y,image_y,album_y,track,date_y,tags
0,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'mbid': '', '#text': 'Pray For Paris'}",$500 Ounces (feat. Freddie Gibbs & Roc Marciano),https://www.last.fm/music/Westside+Gunn/_/$500...,"{'uts': '1705186913', '#text': '13 Jan 2024, 2...",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Pray For Paris,$500 Ounces (feat. Freddie Gibbs & Roc Marciano),"13 Jan 2024, 23:01","MySpotigramBot, underground hip-hop, abstract ..."
1,"{'mbid': '', '#text': 'Westside Gunn'}","[{'size': 'small', '#text': 'https://lastfm.fr...","{'mbid': '', '#text': 'Hall & Nash 2'}",Fuck & Get High,https://www.last.fm/music/Westside+Gunn/_/Fuck...,"{'uts': '1705186705', '#text': '13 Jan 2024, 2...",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hall & Nash 2,Fuck & Get High,"13 Jan 2024, 22:58",
2,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': 'd9972546-40c4-4c3e-be9f-8e9779ea59a2...,Why I do em Like that (feat. Billie Essco),https://www.last.fm/music/Westside+Gunn/_/Why+...,"{'uts': '1705186464', '#text': '13 Jan 2024, 2...",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hitler Wears Hermes 8: Side B,Why I do em Like that (feat. Billie Essco),"13 Jan 2024, 22:54","MySpotigramBot, -1001854528204"
3,{'mbid': 'a536dd07-131b-48ef-ab47-141b752ba082...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': '2488e0ce-7a41-46a2-b1dc-a3fefe38b7d0...,"The Rivington (feat. Conway, Westside Gunn & J...",https://www.last.fm/music/Free+Nationals/_/The...,"{'uts': '1705173308', '#text': '13 Jan 2024, 1...",Free Nationals,"[{'size': 'small', '#text': 'https://lastfm.fr...",Free Nationals,"The Rivington (feat. Conway, Westside Gunn & J...","13 Jan 2024, 19:15",
4,{'mbid': '9f42ba3d-3cfc-499c-ba16-7226f56b622f...,"[{'size': 'small', '#text': 'https://lastfm.fr...",{'mbid': '69f96a35-ddb2-4e26-83e0-2bd6c43776df...,Mamas PrimeTime,https://www.last.fm/music/Westside+Gunn/_/Mama...,"{'uts': '1705173001', '#text': '13 Jan 2024, 1...",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",And Then You Pray for Me,Mamas PrimeTime,"13 Jan 2024, 19:10",


In [ ]:
full_tracks=full_tracks.drop(['artist_x','album_x','name','date_y','image_x'],axis=1)
full_tracks['date_x'] = full_tracks['date_x'].apply(lambda x: x['#text'])
full_tracks.head()

,url,date_x,artist_y,image_y,album_y,track,tags
0,https://www.last.fm/music/Westside+Gunn/_/$500...,"13 Jan 2024, 23:01",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Pray For Paris,$500 Ounces (feat. Freddie Gibbs & Roc Marciano),"MySpotigramBot, underground hip-hop, abstract ..."
1,https://www.last.fm/music/Westside+Gunn/_/Fuck...,"13 Jan 2024, 22:58",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hall & Nash 2,Fuck & Get High,
2,https://www.last.fm/music/Westside+Gunn/_/Why+...,"13 Jan 2024, 22:54",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",Hitler Wears Hermes 8: Side B,Why I do em Like that (feat. Billie Essco),"MySpotigramBot, -1001854528204"
3,https://www.last.fm/music/Free+Nationals/_/The...,"13 Jan 2024, 19:15",Free Nationals,"[{'size': 'small', '#text': 'https://lastfm.fr...",Free Nationals,"The Rivington (feat. Conway, Westside Gunn & J...",
4,https://www.last.fm/music/Westside+Gunn/_/Mama...,"13 Jan 2024, 19:10",Westside Gunn,"[{'size': 'small', '#text': 'https://lastfm.fr...",And Then You Pray for Me,Mamas PrimeTime,


In [ ]:
full_tracks=full_tracks.rename(columns={'artist_y':'artist', 'album_y':'album', 'date_x':'date', 'image_y':'image'})

In [ ]:
full_tracks=full_tracks.astype(str)

In [ ]:
print(full_tracks.dtypes)

url       object
date      object
artist    object
image     object
album     object
track     object
tags      object
dtype: object


In [ ]:
import datetime
full_tracks['date'] = full_tracks['date'].apply(lambda x: datetime.datetime.strptime(x, '%d %b %Y, %H:%M'))


In [ ]:
print(full_tracks.dtypes)

url               object
date      datetime64[ns]
artist            object
image             object
album             object
track             object
tags              object
dtype: object


In [ ]:
import ast
def extract_medium_link(row):
    try:
        row_list = ast.literal_eval(row)  # Safely convert the string to a list
        for item in row_list:
            if item['size'] == 'medium':
                return item['#text']
    except (ValueError, SyntaxError):
        pass  # Handle the case where the data can't be parsed

    return None  # Return None if a medium-sized image is not found

full_tracks['image'] = full_tracks['image'].apply(extract_medium_link)
full_tracks.head()

,url,date,artist,image,album,track,tags
0,https://www.last.fm/music/Westside+Gunn/_/$500...,2024-01-13 23:01:00,Westside Gunn,https://lastfm.freetls.fastly.net/i/u/64s/45b0...,Pray For Paris,$500 Ounces (feat. Freddie Gibbs & Roc Marciano),"MySpotigramBot, underground hip-hop, abstract ..."
1,https://www.last.fm/music/Westside+Gunn/_/Fuck...,2024-01-13 22:58:00,Westside Gunn,https://lastfm.freetls.fastly.net/i/u/64s/8d79...,Hall & Nash 2,Fuck & Get High,
2,https://www.last.fm/music/Westside+Gunn/_/Why+...,2024-01-13 22:54:00,Westside Gunn,https://lastfm.freetls.fastly.net/i/u/64s/4e0b...,Hitler Wears Hermes 8: Side B,Why I do em Like that (feat. Billie Essco),"MySpotigramBot, -1001854528204"
3,https://www.last.fm/music/Free+Nationals/_/The...,2024-01-13 19:15:00,Free Nationals,https://lastfm.freetls.fastly.net/i/u/64s/0a68...,Free Nationals,"The Rivington (feat. Conway, Westside Gunn & J...",
4,https://www.last.fm/music/Westside+Gunn/_/Mama...,2024-01-13 19:10:00,Westside Gunn,https://lastfm.freetls.fastly.net/i/u/64s/0512...,And Then You Pray for Me,Mamas PrimeTime,


In [ ]:
full_tracks.to_csv('theiratebandit.csv', index=False)